In [8]:
import pandas as pd
import numpy as np


In [9]:
patient_df = pd.read_csv("Patient_Data.csv")
billing_df = pd.read_csv("Billing_Data.csv")


In [10]:
patient_df.info()
billing_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 464.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 248.0 bytes


In [11]:
patient_cleaned = patient_df.drop(
    columns=['ReceptionistID', 'CheckInTime'],
    errors='ignore'
)

patient_cleaned.head()


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [12]:
billing_relevant = patient_cleaned[
    ['PatientID', 'Department', 'Doctor', 'BillAmount']
]

billing_relevant.head()


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [13]:
billing_relevant = billing_relevant.drop_duplicates(
    subset='PatientID'
)

billing_relevant.shape


(5, 4)

In [14]:
billing_relevant['BillAmount'] = billing_relevant['BillAmount'].fillna(
    billing_relevant['BillAmount'].mean()
)


In [15]:
dept_bill = billing_relevant.groupby('Department')['BillAmount'].sum()
dept_bill


Department
Cardiology     11200.000000
Dermatology     6233.333333
Neurology       6233.333333
Orthopedics     7500.000000
Name: BillAmount, dtype: float64

In [16]:
merged_df = pd.merge(
    patient_cleaned,
    billing_relevant,
    on=['PatientID', 'Department', 'Doctor', 'BillAmount'],
    how='inner'
)

merged_df.head()


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,103,Charlie,Orthopedics,Dr. Lee,7500.0
2,104,David,Cardiology,Dr. Smith,6200.0
3,101,Alice,Cardiology,Dr. Smith,5000.0


In [17]:
new_patients = pd.DataFrame({
    'PatientID': [9991, 9992],
    'Name': ['Rahul', 'Sneha'],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. Smith', 'Dr. John'],
    'BillAmount': [15000, 18000]
})

updated_df = pd.concat(
    [billing_relevant, new_patients[['PatientID','Department','Doctor','BillAmount']]],
    axis=0,
    ignore_index=True
)

updated_df.tail()


,PatientID,Department,Doctor,BillAmount
2,103,Orthopedics,Dr. Lee,7500.000000
3,104,Cardiology,Dr. Smith,6200.000000
4,105,Dermatology,Dr. Rose,6233.333333
5,9991,Cardiology,Dr. Smith,15000.000000
6,9992,Neurology,Dr. John,18000.000000


In [18]:
extra_cols = pd.DataFrame({
    'InsuranceCovered': [True] * len(updated_df),
    'FinalAmount': updated_df['BillAmount'] * 0.8
})

final_df = pd.concat(
    [updated_df, extra_cols],
    axis=1
)

final_df.head()


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.000000,True,4000.000000
1,102,Neurology,Dr. John,6233.333333,True,4986.666667
2,103,Orthopedics,Dr. Lee,7500.000000,True,6000.000000
3,104,Cardiology,Dr. Smith,6200.000000,True,4960.000000
4,105,Dermatology,Dr. Rose,6233.333333,True,4986.666667


In [19]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         7 non-null      int64  
 1   Department        7 non-null      object 
 2   Doctor            7 non-null      object 
 3   BillAmount        7 non-null      float64
 4   InsuranceCovered  7 non-null      bool   
 5   FinalAmount       7 non-null      float64
dtypes: bool(1), float64(2), int64(1), object(2)
memory usage: 415.0+ bytes
